<a href="https://colab.research.google.com/github/NaHyeonMaeng/CODE_Practice/blob/main/Seq2Seq_%EB%AA%A8%EB%8D%B8%EB%A1%9C_%EC%B1%97%EB%B4%87_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

corpus = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')
corpus.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [6]:
# 챗봇 데이터 샘플 확인
display(corpus['Q'].head()) #질문
print('==='*10)
display(corpus['A'].head()) #답변

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object

0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object

In [5]:
# text와 pair가 쌍으로 이루어짐
print("Q:", corpus['Q'][0])  #하나의 문장을 확인하고 싶을 땐 'x[index]' 활용
print("A:", corpus['A'][0])

Q: 12시 땡!
A: 하루가 또 가네요.


데이터 전처리

In [7]:
# 전체 코퍼스 크기 (11,823개)
corpus.shape[0]

11823

In [8]:
# 2,000개 데이터 셋만 활용 (Google Colab 일 경우 3,000개에서는 메모리 오버되는 현상 발생)
texts = []
pairs = []
for i, (text, pair) in enumerate(zip(corpus['Q'], corpus['A'])):
    texts.append(text)
    pairs.append(pair)
    if i >= 2000:
        break

In [9]:
# question와 answer 데이터 확인
list(zip(texts, pairs))[1995:2000]  #끝부분 샘플 5개

[('배 아프다', '약이 필요하면 도움을 받아보세요.'),
 ('배 터지겠네', '위를 좀 쉬게 해주세요.'),
 ('배 터지겠다.', '산책 좀 해야겠네여.'),
 ('배가 너무 고파', '뭐 좀 챙겨드세요.'),
 ('배가 넘넘 고파', '저도 밥 먹고 싶어요')]

In [10]:
# 데이터 전처리 함수 (정규식(regex expression)활용)
import re  #파이썬에서는 re 모듈을 통해 정규표현식을 사용
def clean_sentence(sentence):
    # 한글, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

In [11]:
# 전처리 함수 테스트
print(clean_sentence('안녕하세요~:)'))
print(clean_sentence('텐서플로^@^%#@!'))

안녕하세요
텐서플로


한글 형태소 분석기 (Konlpy)

In [12]:
# konlpy 설치
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.8 MB/s eta 0:00:00


In [13]:
# Okt형태소
from konlpy.tag import Okt
okt = Okt()
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))  #매개변수를 합침

In [14]:
# 문장 전처리 함수 정의
def clean_and_morph(sentence, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)
    # 형태소 변환
    sentence = process_morph(sentence)
    # Question 인 경우, Answer인 경우를 분기하여 처리합니다.
    if is_question:
        return sentence
    else:
        # START 토큰은 decoder input에 END 토큰은 decoder output에 추가합니다.
        return ('<START> ' + sentence, sentence + ' <END>')

def preprocess(texts, pairs):
    questions = []
    answer_in = []
    answer_out = []

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, is_question=True)
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)

    return questions, answer_in, answer_out

# 샘플 출력
questions, answer_in, answer_out = preprocess(texts, pairs)
print(questions[:2])
print(answer_in[:2])
print(answer_out[:2])

['12시 땡', '1 지망 학교 떨어졌어']
['<START> 하루 가 또 가네요', '<START> 위로 해 드립니다']
['하루 가 또 가네요 <END>', '위로 해 드립니다 <END>']


In [15]:
# 전체 문장을 하나의 리스트로 만들기
all_sentences = questions + answer_in + answer_out

토큰화

In [16]:
# 라이브러리 불어오기
import numpy as np
import warnings
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# WARNING 무시
warnings.filterwarnings('ignore')

In [17]:
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentences)

In [18]:
# 단어사전 확인
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t -> \t{idx}')
    if idx > 10:
        break

<OOV>	 -> 	1
<START>	 -> 	2
<END>	 -> 	3
이	 -> 	4
을	 -> 	5
거	 -> 	6
가	 -> 	7
예요	 -> 	8
도	 -> 	9
해보세요	 -> 	10
요	 -> 	11


In [19]:
# 토큰 갯수 확인
len(tokenizer.word_index)

3604

치환: 텍스트를 시퀀스로 인코딩 (texts_to_sequences)

In [21]:
# 치환: 텍스트를 시퀀스로 인코딩 (texts_to_sequences)
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

# 문장의 길이 맞추기 (pad_sequences)
MAX_LENGTH = 30
question_padded = pad_sequences(question_sequence,
                                maxlen=MAX_LENGTH,
                                truncating='post',  #뒷부분 자름
                                padding='post')  #뒤에 0 채움
answer_in_padded = pad_sequences(answer_in_sequence,
                                 maxlen=MAX_LENGTH,
                                 truncating='post',
                                 padding='post')
answer_out_padded = pad_sequences(answer_out_sequence,
                                  maxlen=MAX_LENGTH,
                                  truncating='post',
                                  padding='post')

모델

In [22]:
# 라이브러리 로드
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

학습용 인코더 (Encoder)

In [23]:
# 인코더
class Encoder(tf.keras.Model):  #tf.keras.Model 상속받음
    def __init__(self, units, vocab_size, embedding_dim, time_steps):  #4개의 파라미터
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        return [hidden_state, cell_state]

학습용 디코더 (Decoder)

In [24]:
# 디코더
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True,
                        )
        self.dense = Dense(vocab_size, activation='softmax')

    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        x = self.dense(x)
        return x, hidden_state, cell_state

모델 결합

In [26]:
# 모델 결합
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            context_vector = self.encoder(encoder_inputs)  #인코더의 출력 결과를 context vector로 저장
            decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs,
                                                 initial_state=context_vector)  #은닉상태와 셀상태는 훈련에 필요 없음
            return decoder_outputs
        else:
            context_vector = self.encoder(inputs)  #인코더의 출력 결과를 context vector로 저장
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(target_seq,
                                                                            initial_state=context_vector)  #디코더에 입력문장과 맥락벡터를 함께 입력값으로 넣음
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1),
                                         dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)

                if decoder_output == self.end_token:
                    break

                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

단어별 원핫인코딩 적용

In [27]:
VOCAB_SIZE = len(tokenizer.word_index)+1

In [28]:
def convert_to_one_hot(padded):
    # 원핫인코딩 초기화
    one_hot_vector = np.zeros((len(answer_out_padded),
                               MAX_LENGTH,
                               VOCAB_SIZE))

    # 디코더 목표(출력)를 원핫인코딩으로 변환
    # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
    for i, sequence in enumerate(answer_out_padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1

    return one_hot_vector

answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)

In [29]:
# 변환된 index를 다시 단어로 변환
def convert_index_to_text(indexs, end_token):

    sentence = ''

    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == end_token:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''

        # 빈칸 추가
        sentence += ' '
    return sentence

학습 (Training)

In [30]:
# 하이퍼 파라미터 정의
BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']

UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
NUM_EPOCHS = 20

In [31]:
# 체크포인트 생성
checkpoint_path = 'model/seq2seq-chatbot-no-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1
                            )

In [32]:
# seq2seq
seq2seq = Seq2Seq(UNITS,
                  VOCAB_SIZE,
                  EMBEDDING_DIM,
                  TIME_STEPS,
                  START_TOKEN,
                  END_TOKEN)

seq2seq.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['acc'])

In [33]:
#문장을 예측하는 함수 정의
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)  #output을 텐서플로 객체 그대로 출력
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)  #어떤 값이 되어도 관계x
    return results

In [34]:
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))

        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)

        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
126/126 [==============================] - ETA: 0s - loss: 2.4487 - acc: 0.7929
Epoch 1: loss improved from inf to 2.44872, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
126/126 [==============================] - 32s 139ms/step - loss: 2.4487 - acc: 0.7929
Epoch 2/10
126/126 [==============================] - ETA: 0s - loss: 1.1961 - acc: 0.8195
Epoch 2: loss improved from 2.44872 to 1.19608, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
126/126 [==============================] - 4s 30ms/step - loss: 1.1961 - acc: 0.8195
Epoch 3/10
126/126 [==============================] - ETA: 0s - loss: 1.1064 - acc: 0.8371
Epoch 3: loss improved from 1.19608 to 1.10638, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
126/126 [==============================] - 2s 18ms/step - loss: 1.1064 - acc: 0.8371
Epoch 4/10
126/126 [==============================] - ETA: 0s - loss: 1.0582 - acc: 0.8396
Epoch 4: loss i

예측

In [35]:
# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

make_question('오늘 날씨 어때?')

array([[124, 170, 347,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]], dtype=int32)

In [36]:
# 챗봇
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(seq2seq, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

# 챗봇 실행
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))

<< 말을 걸어 보세요!
안녕
>> 챗봇 응답: 약 바르고 얼른 나으세요 
<< 말을 걸어 보세요!
아니 안녕이라고
>> 챗봇 응답: 자신 의 감정 을 주변 사람 들 에게 터 놓고 이야기 해보세요 
<< 말을 걸어 보세요!
어이없네
>> 챗봇 응답: 약 바르고 얼른 나으세요 
<< 말을 걸어 보세요!
뭐해
>> 챗봇 응답: 남 들 눈 은 신경 쓰지 마세요 
<< 말을 걸어 보세요!
요즘 고민이있어
>> 챗봇 응답: 자신 하고 만나는 시간 도 중요해요 
<< 말을 걸어 보세요!
공부할까?
>> 챗봇 응답: 자신 만 행복하면 돼요 
<< 말을 걸어 보세요!
넌 행복해?
>> 챗봇 응답: 자신 만 행복하면 돼요 
<< 말을 걸어 보세요!
q


# **Seq2Seq with Attention**

In [36]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import plot_model

In [37]:
# 인코더
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps,
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True,
                         name='LSTM')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

In [38]:
# 디코더
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size,
                                   embedding_dim,
                                   input_length=time_steps,
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        self.lstm = LSTM(units,
                         return_state=True,
                         return_sequences=True,
                         name='LSTM'
                        )
        self.attention = Attention(name='Attention')
        self.dense = Dense(vocab_size,
                           activation='softmax',
                           name='Dense')

    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)

        # (attention) key_value, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 vector를 생성합니다.
        key_value = tf.concat([initial_state[0][:, tf.newaxis, :],
                               x[:, :-1, :]], axis=1)
        # 이전 hidden_state의 값을 concat으로 만든 vector와 encoder에서 나온
        # 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention([key_value, encoder_inputs])
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)

        x = self.dense(x)
        return x, hidden_state, cell_state

In [39]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)


    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            # (attention) decoder 입력 값 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs),
                                                 initial_state=context_vector)
            return decoder_outputs
        else:
            x = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(x)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq),
                                                                            initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)

                if decoder_output == self.end_token:
                    break

                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

In [40]:
# 체크포인트
checkpoint_path = 'model/seq2seq-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1
                            )

In [41]:
# 훈련
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))

        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)

        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
126/126 [==============================] - ETA: 0s - loss: 0.1080 - acc: 0.9724
Epoch 1: loss improved from inf to 0.10801, saving model to model/seq2seq-attention-checkpoint.ckpt
126/126 [==============================] - 5s 36ms/step - loss: 0.1080 - acc: 0.9724
Epoch 2/10
126/126 [==============================] - ETA: 0s - loss: 0.1076 - acc: 0.9716
Epoch 2: loss improved from 0.10801 to 0.10756, saving model to model/seq2seq-attention-checkpoint.ckpt
126/126 [==============================] - 3s 22ms/step - loss: 0.1076 - acc: 0.9716
Epoch 3/10
124/126 [============================>.] - ETA: 0s - loss: 0.1055 - acc: 0.9723
Epoch 3: loss improved from 0.10756 to 0.10547, saving model to model/seq2seq-attention-checkpoint.ckpt
126/126 [==============================] - 2s 14ms/step - loss: 0.1055 - acc: 0.9722
Epoch 4/10
125/126 [============================>.] - ETA: 0s - loss: 0.1033 - acc: 0.9729
Epoch 4: loss improved from 0.10547 to 0.10329, sa

In [42]:
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))

<< 말을 걸어 보세요!
안녕
>> 챗봇 응답: 독서 와 음악 감상 이라고 하고 싶지만 아무 것 도 안 했어요 
<< 말을 걸어 보세요!
아니 안녕이라고
>> 챗봇 응답: 화장실 가세 요 
<< 말을 걸어 보세요!
오늘 너무 행복해
>> 챗봇 응답: 즐거운 시작 되길 바랍니다 
<< 말을 걸어 보세요!
조금 심심해
>> 챗봇 응답: 친구 들 과 연락 해보세요 
<< 말을 걸어 보세요!
배고픈데 뭐 먹을까?
>> 챗봇 응답: 좀 먹어도 괜찮아요 
<< 말을 걸어 보세요!
아니 메뉴 추천좀 해줘
>> 챗봇 응답: 독서 와 음악 감상 이라고 하고 싶지만 아무 것 도 안 했어요 
<< 말을 걸어 보세요!
메뉴 추천 해줘
>> 챗봇 응답: 내 집 마련 축하 드려요 
<< 말을 걸어 보세요!
나 집 없어...
>> 챗봇 응답: 냉장고 파먹기 해보세요 
<< 말을 걸어 보세요!
q
